# Define the level 1 models

In [83]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import data_prep

# Hyperparameters
timesteps = 10  # Number of time steps
num_features = 6  # Number of features
n_epochs = 100  # Number of epochs
learning_rate = 0.001

# LSTM Model
class LSTMModel(nn.Module):
    def __init__(self):
        super(LSTMModel, self).__init__()
        self.lstm1 = nn.LSTM(num_features, 60, batch_first=True)
        self.dropout1 = nn.Dropout(0.4)
        self.lstm2 = nn.LSTM(60, 55, batch_first=True)
        self.dropout2 = nn.Dropout(0.4)
        self.lstm3 = nn.LSTM(55, 50, batch_first=True)
        self.dropout3 = nn.Dropout(0.4)
        self.lstm4 = nn.LSTM(50, 45, batch_first=True)  
        self.dropout4 = nn.Dropout(0.4)
        self.fc = nn.Linear(45, 1) 
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x, _ = self.lstm1(x)
        x = self.dropout1(x)
        x, _ = self.lstm2(x)
        x = self.dropout2(x)
        x, _ = self.lstm3(x)
        x = self.dropout3(x)
        x, _ = self.lstm4(x)
        x = self.dropout4(x)
        x = x[:, -1, :]  # Take the last output from the last LSTM layer
        x = self.fc(x)   # Linear layer to map to 1 output
        x = self.sigmoid(x)
        return x

# GRU Model
class GRUModel(nn.Module):
    def __init__(self):
        super(GRUModel, self).__init__()
        self.gru1 = nn.GRU(num_features, 60, batch_first=True)
        self.dropout1 = nn.Dropout(0.2)
        self.gru2 = nn.GRU(60, 55, batch_first=True)
        self.dropout2 = nn.Dropout(0.2)
        self.gru3 = nn.GRU(55, 50, batch_first=True)
        self.dropout3 = nn.Dropout(0.2)
        self.gru4 = nn.GRU(50, 45, batch_first=True)
        self.dropout4 = nn.Dropout(0.2)
        self.fc = nn.Linear(45, 1)  # Ensures the output is of size [batch_size, 1]
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x, _ = self.gru1(x)
        x = self.dropout1(x)
        x, _ = self.gru2(x)
        x = self.dropout2(x)
        x, _ = self.gru3(x)
        x = self.dropout3(x)
        x, _ = self.gru4(x)
        x = self.dropout4(x)
        x = x[:, -1, :]  # Take the last output
        x = self.fc(x)   # Linear layer to map to 1 output
        x = self.sigmoid(x)
        return x

# Define the meta learner

In [84]:
# it's a fully-connect neuralnetwork with three layers; the activation function for this model is the Rectified Linear Unit (ReLu).
# NOTE: The paper doesn't specify the number of neurons in the hidden layers, so I'm basing on the stanford paper
class MetaLearner(nn.Module):
    def __init__(self):
        super(MetaLearner, self).__init__()
        self.fc1 = nn.Linear(2, 30)
        self.fc2 = nn.Linear(30, 25)
        self.fc3 = nn.Linear(25, 20)
        self.fc4 = nn.Linear(20, 1)
        self.sigmoid = nn.Sigmoid() 
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        x = self.sigmoid(x) 
        return x

# Load in data

In [85]:
import pandas as pd
import numpy as np
from os.path import join

split = 0.69  # Adjust to allocate space for validation set
val_split = 0.16  # 15% for validation, and implicitly 15% for test due to remaining percentage
sequence_length = 10
normalise = True
batch_size = 100
input_dim = 7
input_timesteps = 9
neurons = 50
epochs = 5
prediction_len = 1
dense_output = 1
drop_out = 0

#filepath = 'data/original_dataset/source_price.csv'
filepath = 'data/original_dataset/amzn_source_price.csv'

# This approach does not normalize the compound scores in the range of 0 to 1
def get_data_old(filepath: str):
  # Load data, modify cols whenever necessary
  dataframe = pd.read_csv(filepath)

  #cols = ['Adj Close', 'wsj_mean_compound', 'cnbc_mean_compound', 'fortune_mean_compound', 'reuters_mean_compound']
  cols = ['Adj Close', 'mean_compound_reuters', 'mean_compound_guardian', 'mean_compound_cnbc', 'mean_compound_other', 'mean_compound_stocktwits', 'mean_compound_twitter']

  len_dataframe = dataframe.shape[0]

  # Split data into train, validation, and test
  i_split = int(len(dataframe) * split)
  i_val = int(len(dataframe) * (split + val_split))

  data_train = dataframe.get(cols).values[:i_split]
  data_val = dataframe.get(cols).values[i_split:i_val]
  data_test = dataframe.get(cols).values[i_val:]

  # print(data_train[0:5,0])

  len_train = len(data_train)
  len_val = len(data_val)
  len_test = len(data_test)
  len_train_windows = None

  # Process train data
  data_windows = []
  for i in range(len_train - sequence_length):
      data_windows.append(data_train[i:i+sequence_length])
  data_windows = np.array(data_windows).astype(float)
  window_data = data_windows
  win_num = window_data.shape[0]
  col_num = window_data.shape[2]
  normalised_data = []
  record_min = []
  record_max = []

  # Normalize train data
  for win_i in range(win_num):
      normalised_window = []
      for col_i in range(0,1):
        temp_col = window_data[win_i,:,col_i]
        temp_min = min(temp_col)
        record_min.append(temp_min)
        temp_col = temp_col - temp_min
        temp_max = max(temp_col)
        record_max.append(temp_max)
        temp_col = temp_col / temp_max
        normalised_window.append(temp_col)
      for col_i in range(1,col_num):
        normalised_window.append(window_data[win_i,:,col_i])
      normalised_window = np.array(normalised_window).T
      normalised_data.append(normalised_window)
  normalised_data = np.array(normalised_data)
  x_train = normalised_data[:, :-1]
  # Classification problem now
  y_train = []
  for i in range(len_train - sequence_length):
      current_last = data_train[i+sequence_length-1, 0]
      next_first = data_train[i+sequence_length, 0]
      y_train.append(1 if next_first > current_last else 0)
  y_train = np.array(y_train)

  # Process validation data
  data_windows = []
  for i in range(len_val - sequence_length):
      data_windows.append(data_val[i:i+sequence_length])
  data_windows = np.array(data_windows).astype(float)
  window_data = data_windows
  win_num = window_data.shape[0]
  normalised_data = []

  # Normalize validation data
  for win_i in range(win_num):
      normalised_window = []
      for col_i in range(0,1):
        temp_col = window_data[win_i,:,col_i]
        temp_min = min(temp_col)
        temp_col = temp_col - temp_min
        temp_max = max(temp_col)
        temp_col = temp_col / temp_max
        normalised_window.append(temp_col)
      for col_i in range(1,col_num):
        normalised_window.append(window_data[win_i,:,col_i])
      normalised_window = np.array(normalised_window).T
      normalised_data.append(normalised_window)
  normalised_data = np.array(normalised_data)
  x_val = normalised_data[:, :-1]
  y_val = []
  for i in range(len_val - sequence_length):
      current_last = data_val[i+sequence_length-1, 0]
      next_first = data_val[i+sequence_length, 0]
      y_val.append(1 if next_first > current_last else 0)
  y_val = np.array(y_val)

  # Process test data
  data_windows = []
  for i in range(len_test - sequence_length):
      data_windows.append(data_test[i:i+sequence_length])
  data_windows = np.array(data_windows).astype(float)
  y_test_ori = data_windows[:, -1, [0]]
  window_data = data_windows
  win_num = window_data.shape[0]
  normalised_data = []

  # Normalize test data
  for win_i in range(win_num):
      normalised_window = []
      for col_i in range(0,1):
        temp_col = window_data[win_i,:,col_i]
        temp_min = min(temp_col)
        temp_col = temp_col - temp_min
        temp_max = max(temp_col)
        temp_col = temp_col / temp_max
        normalised_window.append(temp_col)
      for col_i in range(1,col_num):
        normalised_window.append(window_data[win_i,:,col_i])
      normalised_window = np.array(normalised_window).T
      normalised_data.append(normalised_window)
  normalised_data = np.array(normalised_data)
  x_test = normalised_data[:, :-1]
  y_test = []
  for i in range(len_test - sequence_length):
      current_last = data_test[i+sequence_length-1, 0]
      next_first = data_test[i+sequence_length, 0]
      y_test.append(1 if next_first > current_last else 0)
  y_test = np.array(y_test)
  return x_train, y_train, x_val, y_val, x_test, y_test

def get_data_normal(filepath: str):
  df = pd.read_csv(filepath)
  # Partition data into training, validation and test sets. Training data should be from date 12/07/2017 to 04/09/2018, validation data (from 04/10/2018 to 05/04/2018), and test data (from 05/07/2018 to 06/01/2018)

  from sklearn.preprocessing import MinMaxScaler

  # Count entries in 'date' column
  num_entries = df['date'].count()

  # Calculate indices for train, validation, and test splits
  num_train = int(0.7 * num_entries)
  num_val = int(0.15 * num_entries)
  num_test = num_entries - num_train - num_val  # Ensuring all rows are included

  # Split the data
  df_train = df.loc[:num_train - 1]
  df_val = df.loc[num_train:num_train + num_val - 1]
  df_test = df.loc[num_train + num_val:]

  # Hardcodidly extracting the exact dates for the partitioning
  # '''df_train = df.loc[0:82]
  # df_val = df.loc[83:101]
  # df_test = df.loc[102:]'''

  # print(df_val.head())
  # print(df_val.tail())
  # print(df_test.head())
  # print(df_val)

  df_train = df_train.drop(columns=['date'])
  df_val = df_val.drop(columns=['date'])
  df_test = df_test.drop(columns=['date'])

  sc = MinMaxScaler(feature_range=(0,1))
  print("DF TRAIN", df_train.head())
  df_train = sc.fit_transform(df_train)
  df_val = sc.transform(df_val)
  df_test = sc.transform(df_test)

  def create_sequences_numpy_classification(data, n_days):
      X, y = [], []
      for i in range(n_days, len(data) - 1): 
          # print("X")
          X.append(data[i-n_days:i])
          # print(data[i-n_days:i])
          y.append(1 if data[i][-1] - data[i-1][-1] > 0 else 0) #Classification task
          # print(data[i][-1], data[i-1][-1])
          # print("Y")
          # print(y[-1])
          
      # Delete the first column of X
      # X = np.delete(X, 0, axis=2)
      return np.array(X), np.array(y)

  x_train, y_train = create_sequences_numpy_classification(df_train, 10)
  x_val, y_val = create_sequences_numpy_classification(df_val, 10)
  x_test, y_test = create_sequences_numpy_classification(df_test, 10)
  return x_train, y_train, x_val, y_val, x_test, y_test

def get_data_stanford():
    DATA_PATH = join('data', 'original_dataset', 'cleaned_tesla_data.csv')

    HORIZON = 10
    DAYS_FORWARD = 1
    END_SPLIT = 40


    # split_y: tuple
    #     (train_y, validate_y, test_y)
    # split_X: tuple
    #     (train_X, validate_X, test_X)
    split_y, split_X = data_prep.data_prep(DATA_PATH, HORIZON, DAYS_FORWARD, END_SPLIT)
    return split_X, split_y

# x_train, y_train, x_val, y_val, x_test, y_test = get_data_normal(filepath)

(x_train, x_val, x_test), (y_train, y_val, y_test) = get_data_stanford()

# print("train X", split_X[0].shape)
# print("val X", split_X[1].shape)
# print("test X", split_X[2].shape)
# print("train y", split_y[0].shape)
# print("val y", split_y[1].shape)
# print("test y", split_y[2].shape)
# print("NEW")

print('x_train.shape', x_train.shape)
print('y_train.shape', y_train.shape)
print('x_val.shape', x_val.shape)
print('y_val.shape', y_val.shape)
print('x_test.shape', x_test.shape)
print('y_test.shape', y_test.shape)



# print(x_train[0])

# print(y_train[0:5])

# print(x_val[0])

# print(y_val[0])

# print(x_test[0])

# print(y_test[0])


x_train.shape (666, 10, 6)
y_train.shape (666,)
x_val.shape (40, 10, 6)
y_val.shape (40,)
x_test.shape (40, 10, 6)
y_test.shape (40,)


# Instatiate the models

In [86]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
# Instantiate models
lstm_model = LSTMModel()
gru_model = GRUModel()
meta_model = MetaLearner()

# Define loss and optimizer
criterion = nn.BCELoss()
lstm_optimizer = optim.RMSprop(lstm_model.parameters(), lr=0.0008) # 16 batch size, 150 epochs
gru_optimizer = optim.RMSprop(gru_model.parameters(), lr=0.0008) # 16 batch size, 200 epochs
base_models_batch_size = 16
meta_optimizer = optim.Adam(meta_model.parameters(), lr=0.001) # 100 epochs, 8 batch size
meta_learner_batch_size = 8

# Train the base models

In [87]:
# Convert data to PyTorch tensors and create DataLoader
X_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

print(X_train_tensor.shape)
print(y_train_tensor.shape)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=base_models_batch_size, shuffle=True) #Stanford had shuffle true

# Training function
def train_model(model, optimizer, criterion, train_loader, n_epochs):
    model.train()
    for epoch in range(n_epochs):
        epoch_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            output = model(X_batch)
            loss = criterion(output, y_batch.view(-1, 1))
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f'Epoch {epoch+1}/{n_epochs}, Loss: {epoch_loss/len(train_loader)}')

# Train the LSTM model
print("Training LSTM Model")
train_model(lstm_model, lstm_optimizer, criterion, train_loader, 150)

# Train the GRU model
print("Training GRU Model")
train_model(gru_model, gru_optimizer, criterion, train_loader, 200)

torch.Size([666, 10, 6])
torch.Size([666])
Training LSTM Model


Epoch 1/150, Loss: 0.6925367911656698
Epoch 2/150, Loss: 0.6896093218099504
Epoch 3/150, Loss: 0.6927688320477804
Epoch 4/150, Loss: 0.6925106531097776
Epoch 5/150, Loss: 0.693042591923759
Epoch 6/150, Loss: 0.6917174756526947
Epoch 7/150, Loss: 0.6936698172773633
Epoch 8/150, Loss: 0.6914670836357844
Epoch 9/150, Loss: 0.6930848033655257
Epoch 10/150, Loss: 0.6928743053050268
Epoch 11/150, Loss: 0.6920885528836932
Epoch 12/150, Loss: 0.6916447764351255
Epoch 13/150, Loss: 0.6930122063273475
Epoch 14/150, Loss: 0.6933688450427282
Epoch 15/150, Loss: 0.691185264360337
Epoch 16/150, Loss: 0.6923364358288902
Epoch 17/150, Loss: 0.6917492293176197
Epoch 18/150, Loss: 0.6926387094316029
Epoch 19/150, Loss: 0.6920432987667265
Epoch 20/150, Loss: 0.6926585379100981
Epoch 21/150, Loss: 0.6912643285024733
Epoch 22/150, Loss: 0.6925120367890313
Epoch 23/150, Loss: 0.690949349176316
Epoch 24/150, Loss: 0.6913746623765855
Epoch 25/150, Loss: 0.6925936738650004
Epoch 26/150, Loss: 0.692194034655888

# Use base models to predict the validation data, this will be used as input to the Meta Learner

In [88]:
lstm_val_predictions = lstm_model(torch.tensor(x_val, dtype=torch.float32)).detach().numpy().reshape(-1,1)
gru_val_predictions = gru_model(torch.tensor(x_val, dtype=torch.float32)).detach().numpy().reshape(-1,1)

# lstm_pred = lstm_model.predict(X).reshape(-1, 1)
# gru_pred = gru_model.predict(X).reshape(-1, 1)

# Form and return new data set
# new_X = np.hstack((lstm_pred, gru_pred))


# Combine predictions to form new training data for the meta-learner
meta_X_train = np.hstack((lstm_val_predictions, gru_val_predictions))#meta_X_train = np.concatenate((lstm_val_predictions, gru_val_predictions), axis=1)

print(meta_X_train.shape)

print(meta_X_train)

print(y_val)

print(x_val)

(40, 2)
[[0.516736   0.46189052]
 [0.52425104 0.48631704]
 [0.52409965 0.50474036]
 [0.535297   0.49517587]
 [0.547378   0.52963537]
 [0.5369737  0.5083322 ]
 [0.5239233  0.49191448]
 [0.47097984 0.50310606]
 [0.47814897 0.48590773]
 [0.49187255 0.47363514]
 [0.4468163  0.43271756]
 [0.44914964 0.47728932]
 [0.46036085 0.49529105]
 [0.44215804 0.503229  ]
 [0.46840578 0.4733225 ]
 [0.46385634 0.49194252]
 [0.46563783 0.50629836]
 [0.49265054 0.4611489 ]
 [0.49618402 0.4764875 ]
 [0.46536583 0.43460697]
 [0.43636358 0.45869818]
 [0.4727506  0.46687996]
 [0.46085545 0.46997508]
 [0.45532218 0.48353973]
 [0.45749515 0.44683164]
 [0.48587045 0.46103504]
 [0.47340906 0.43495625]
 [0.47155273 0.40900496]
 [0.48620322 0.39225423]
 [0.44880024 0.43028203]
 [0.4385189  0.46465787]
 [0.46292764 0.431434  ]
 [0.4419427  0.3802687 ]
 [0.47270033 0.48156247]
 [0.49414676 0.4777984 ]
 [0.463803   0.4817617 ]
 [0.44387853 0.46524623]
 [0.49341908 0.46837053]
 [0.49157387 0.4932608 ]
 [0.47571436 0.49

# Train meta learner

In [89]:
meta_model = MetaLearner()
meta_criterion = nn.BCELoss()
meta_optimizer = optim.Adam(meta_model.parameters(), lr=0.008)

meta_X_train_tensor = torch.tensor(meta_X_train, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

meta_train_dataset = TensorDataset(meta_X_train_tensor, y_val_tensor)
meta_train_loader = DataLoader(meta_train_dataset, batch_size=8, shuffle=False)

train_model(meta_model, meta_optimizer, meta_criterion, meta_train_loader, 100)

Epoch 1/100, Loss: 0.6756232619285584
Epoch 2/100, Loss: 0.6516303896903992
Epoch 3/100, Loss: 0.6331756591796875
Epoch 4/100, Loss: 0.6265263080596923
Epoch 5/100, Loss: 0.6329083502292633
Epoch 6/100, Loss: 0.6360498666763306
Epoch 7/100, Loss: 0.6349086105823517
Epoch 8/100, Loss: 0.6335639119148254
Epoch 9/100, Loss: 0.6326347768306733
Epoch 10/100, Loss: 0.6319206595420838
Epoch 11/100, Loss: 0.6314817726612091
Epoch 12/100, Loss: 0.6314120769500733
Epoch 13/100, Loss: 0.6316119968891144
Epoch 14/100, Loss: 0.6318620920181275


Epoch 15/100, Loss: 0.6320068717002869
Epoch 16/100, Loss: 0.6320198059082032
Epoch 17/100, Loss: 0.6319465339183807
Epoch 18/100, Loss: 0.6318423330783844
Epoch 19/100, Loss: 0.6317469656467438
Epoch 20/100, Loss: 0.6316810250282288
Epoch 21/100, Loss: 0.6316468834877014
Epoch 22/100, Loss: 0.6316346049308776
Epoch 23/100, Loss: 0.631629902124405
Epoch 24/100, Loss: 0.6316215991973877
Epoch 25/100, Loss: 0.6316048085689545
Epoch 26/100, Loss: 0.6315743803977967
Epoch 27/100, Loss: 0.6315429151058197
Epoch 28/100, Loss: 0.6315185368061066
Epoch 29/100, Loss: 0.6314856231212616
Epoch 30/100, Loss: 0.6314568936824798
Epoch 31/100, Loss: 0.6314382433891297
Epoch 32/100, Loss: 0.6314112663269043
Epoch 33/100, Loss: 0.6313881516456604
Epoch 34/100, Loss: 0.6313630223274231
Epoch 35/100, Loss: 0.6313347041606903
Epoch 36/100, Loss: 0.6313071250915527
Epoch 37/100, Loss: 0.6312850594520569
Epoch 38/100, Loss: 0.6312565207481384
Epoch 39/100, Loss: 0.6312348961830139
Epoch 40/100, Loss: 0.6312

In [90]:
from sklearn.metrics import precision_recall_fscore_support
#  the test dataset will be input into the sub-models again to produce intermediate test data for the meta-learner. Afterward, the meta-learner will use the intermediate test predictions from the sub-models to make the final predictions.
lstm_test_predictions = lstm_model(torch.tensor(x_test, dtype=torch.float32)).detach().numpy()
gru_test_predictions = gru_model(torch.tensor(x_test, dtype=torch.float32)).detach().numpy()

meta_X_test = np.concatenate((lstm_test_predictions, gru_test_predictions), axis=1)
meta_X_test_tensor = torch.tensor(meta_X_test, dtype=torch.float32)

meta_test_predictions = meta_model(meta_X_test_tensor).detach().numpy()

# Evaluation metrics
meta_test_predictions = np.round(meta_test_predictions)
accuracy = np.mean(meta_test_predictions == y_test)
print(f'Accuracy: {accuracy}')

precision, recall, f1, _ = precision_recall_fscore_support(y_test, meta_test_predictions, average='binary')
print(f'Precision: {precision}, Recall: {recall}, F1 Score: {f1}')
print("meta predictions", meta_test_predictions)


lstm_test_predictions = np.round(lstm_test_predictions)
accuracy = np.mean(lstm_test_predictions == y_test)
print(f'Accuracy for lstm: {accuracy}')

precision, recall, f1, _ = precision_recall_fscore_support(y_test, lstm_test_predictions, average='binary')
print(f'Precision for lstm: {precision}, Recall for lstm: {recall}, F1 Score for lstm: {f1}')
print("lstm predictions", lstm_test_predictions)

gru_test_predictions = np.round(gru_test_predictions)
accuracy = np.mean(gru_test_predictions == y_test)
print(f'Accuracy for gru: {accuracy}')
print("gru predictions", gru_test_predictions)

precision, recall, f1, _ = precision_recall_fscore_support(y_test, gru_test_predictions, average='binary')
print(f'Precision for gru: {precision}, Recall for gru: {recall}, F1 Score for gru: {f1}')


Accuracy: 0.65
Precision: 0.65, Recall: 1.0, F1 Score: 0.787878787878788
meta predictions [[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
Accuracy for lstm: 0.35
Precision for lstm: 0.0, Recall for lstm: 0.0, F1 Score for lstm: 0.0
lstm predictions [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
Accuracy for gru: 0.365
gru predictions [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
Precision for gru: 0.5, Recall for g

/home/karlo/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
